In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import functools as f
from sklearn.model_selection import train_test_split
import splitfolders
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
import visualkeras

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import splitfolders
splitfolders.ratio('../Alzheimer_classification/Dataset', output="output", seed=1345, ratio=(.8, 0.1,0.1))

In [ ]:
# Height and width in pixels
img_height = 128
img_width = 128

data_splits = []
direct = ['output/train', 'output/test', 'output/val']
for d in direct: 
    var_name = tf.keras.utils.image_dataset_from_directory(
                    d,
                    label_mode='categorical',
                    labels='inferred',
                    batch_size=64,
                    image_size=(img_height, img_width),
                    seed=123,
                )
    data_splits.append(var_name)
    

In [ ]:
data_splits

In [ ]:
#Visualize images
dem_types = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
examples = ['Mild_Demented/mild_2.jpg', 'Moderate_Demented/moderate_2.jpg', 'Non_Demented/non_2.jpg', 'Very_Mild_Demented/verymild_2.jpg']
for i in range(len(dem_types)): 
    plt.plot(128,128)
    plt.figure(i+1) 
    plt.title(dem_types[i])
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    plt.imshow(image.imread('Dataset/'+ examples[i]))
plt.show()

## LeNet-5 CNN Architecture Modified (MaxPool and Dropout 25%)

In [ ]:
# LeNet-5 CNN Architecture Modified (MaxPool and Dropout 25%)
model = keras.Sequential()
model.add(Rescaling(1./255, input_shape=(128, 128, 3)))
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation = 'relu', input_shape = (128, 128, 3), kernel_initializer="he_normal"))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation = 'relu', input_shape = (128, 128, 3), kernel_initializer="he_normal"))
model.add(layers.MaxPooling2D())

model.add(Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(units = 128, activation = 'relu', kernel_initializer="he_normal"))
model.add(layers.Dense(units = 64, activation = 'relu'))
model.add(layers.Dense(units = 4, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
visualkeras.layered_view(model)

In [ ]:
hist = model.fit(data_splits[0], epochs = 20, validation_data = data_splits[2])

In [ ]:
print("Accuracy - LeNet5 Architecture Modified")
plt.plot(hist.history['accuracy'],c='b',label='train - accuracy')
plt.plot(hist.history['val_accuracy'],c='r',label='validation - accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy - LeNet5 Architecture Modified')
plt.show()

print("\n\n")
print('Loss - LeNet5 Architecture Modified')
plt.plot(hist.history['loss'],c='orange',label='train - loss')
plt.plot(hist.history['val_loss'],c='g',label='validation - loss')
plt.legend(loc='upper right')
plt.title('Loss - LeNet5 Architecture Modified')
plt.show()

In [ ]:
model.evaluate(data_splits[1])

In [ ]:
model.save('LeNet-5e50Mod.h5')

In [ ]:
model2 = load_model('LeNet-5e50Mod.h5')
preds = np.array([])
labels =  np.array([])
for x, y in data_splits[1]:
    preds = np.concatenate([preds, np.argmax(model2.predict(x), axis = -1)])
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

print("Accuracy:",metrics.accuracy_score(labels, preds))
print("Sensitivity:",metrics.recall_score(labels, preds, average='macro'))
print("Precision:",metrics.precision_score(labels, preds, average='macro', zero_division=1))
print("Recall:",metrics.recall_score(labels, preds, average='macro' ))
print("F1 Score:",f1_score(labels, preds, average='macro'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

Cm=confusion_matrix(labels, preds)
ax=plt.axes()
sns.heatmap(Cm, cmap='Blues' ,annot=True,
           annot_kws={'size':10},
           ax=ax
           )

plt.title("Confusion Metrics for Alzheimer Disease Classification")
plt.show()

## LeNet-5 CNN Architecture (with MaxPooling)

In [ ]:
#LeNet-5 CNN Architecture (with MaxPooling)
model2 = keras.Sequential()
model2.add(Rescaling(1./255, input_shape=(128, 128, 3)))
model2.add(Conv2D(filters=6, kernel_size=(3, 3), padding='same', activation = 'relu', kernel_initializer="he_normal"))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation = 'relu', kernel_initializer="he_normal"))
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(Dropout(0.20))

model2.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer="he_normal"))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(units = 128, activation = 'relu', kernel_initializer="he_normal"))
model2.add(Dense(units = 64, activation = 'relu'))
model2.add(Dense(units = 4, activation = 'softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
visualkeras.layered_view(model2)

In [ ]:
hist2 = model2.fit(data_splits[0], epochs = 20, validation_data = data_splits[2])

In [ ]:
print("Accuracy - LeNet-5 with MaxPooling")
plt.plot(hist2.history['accuracy'],c='b',label='train - accuracy')
plt.plot(hist2.history['val_accuracy'],c='r',label='validation - accuracy')
plt.legend(loc='lower right')
plt.show()

print("\n\n")
print('Loss - LeNet-5 with MaxPooling')
plt.plot(hist2.history['loss'],c='orange',label='train - loss')
plt.plot(hist2.history['val_loss'],c='g',label='validation - loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
model2.evaluate(data_splits[1])

In [ ]:
model2.save('LeNet-5Maxe50.h5')

In [ ]:
model2 = load_model('LeNet-5Maxe50.h5')
preds = np.array([])
labels =  np.array([])
for x, y in data_splits[1]:
    preds = np.concatenate([preds, np.argmax(model2.predict(x), axis = -1)])
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

print("Accuracy:",metrics.accuracy_score(labels, preds))
print("Sensitivity:",metrics.recall_score(labels, preds, average='macro'))
print("Precision:",metrics.precision_score(labels, preds, average='macro', zero_division=1))
print("Recall:",metrics.recall_score(labels, preds, average='macro' ))
print("F1 Score:",f1_score(labels, preds, average='macro'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

Cm=confusion_matrix(labels, preds)
ax=plt.axes()
sns.heatmap(Cm, cmap='Blues' ,annot=True,
           annot_kws={'size':10},
           ax=ax
           )

plt.title("Confusion Metrics for Alzheimer Disease Classification")
plt.show()

## Custom CNN Architecture

In [ ]:
# New Architecture
model3 = Sequential()
model3.add(Rescaling(1./255, input_shape=(img_height, img_width, 3)))
model3.add(Conv2D(filters=16,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer="he_normal"))
model3.add(MaxPooling2D(pool_size=(2,2)))


model3.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer="he_normal"))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add(Dropout(0.20))

model3.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer="he_normal"))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(128,activation="relu",kernel_initializer="he_normal"))
model3.add(Dense(64,"relu"))
model3.add(Dense(4,"softmax"))
model3.compile(loss="categorical_crossentropy", optimizer = 'adam',metrics=["accuracy"])
visualkeras.layered_view(model3)

In [ ]:
hist3 = model3.fit(data_splits[0], epochs = 20validation_data = data_splits[2])

In [ ]:
print("Accuracy - CNN Model")
plt.plot(hist3.history['accuracy'],c='b',label='train - accuracy')
plt.plot(hist3.history['val_accuracy'],c='r',label='validation - accuracy')
plt.legend(loc='lower right')
plt.show()

print("\n\n")
print('Loss - CNN Model')
plt.plot(hist3.history['loss'],c='orange',label='train - loss')
plt.plot(hist3.history['val_loss'],c='g',label='validation - loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
model3.evaluate(data_splits[1])

In [ ]:
model3.save('CNN_Model.h5')

In [ ]:
cnn = load_model('CNN_Model.h5')
img = load_img("output/test/Very_Mild_Demented/verymild_1737.jpg")
img = img.resize((128, 128))
img = img_to_array(img) 

img = img.reshape(-1,128, 128,3)

print(img.shape)

In [ ]:
print(np.argmax(cnn.predict(img)))

In [ ]:
preds = np.array([])
labels =  np.array([])
for x, y in data_splits[1]:
    preds = np.concatenate([preds, np.argmax(cnn.predict(x), axis = -1)])
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

print("Accuracy:",metrics.accuracy_score(labels, preds))
print("Sensitivity:",metrics.recall_score(labels, preds, average='macro'))
print("Precision:",metrics.precision_score(labels, preds, average='macro', zero_division=1))
print("Recall:",metrics.recall_score(labels, preds, average='macro' ))
print("F1 Score:",f1_score(labels, preds, average='macro'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

Cm=confusion_matrix(labels, preds)
ax=plt.axes()
sns.heatmap(Cm, cmap='Blues' ,annot=True,
           annot_kws={'size':10},
           ax=ax
           )

plt.title("Confusion Metrics for Alzheimer Disease Classification")
plt.show()